In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os
import time
import concurrent.futures
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

/Users/leilayfarsani/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
api_key = os.getenv('IATI_API_KEY')

if not api_key:
    raise ValueError("API key not found. Please make sure it is set in the .env file or update it if necessary.")

Activity/apache/select

In [3]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page(start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity"))',
        'fl': 'iati_identifier,title_narrative,description_narrative,sector_code,activity_date_iso_date,activity_date_type,recipient_country_code',
        'fq': 'activity_date_type:2 AND activity_date_iso_date:[2021-01-01T00:00:00Z TO *]',  
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    for attempt in range(5):  
        try:
            response = requests_retry_session().get(base_url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()['response']['docs']
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                wait = 2 ** attempt  # exponential backoff
                print(f"Rate limit hit. Waiting for {wait} seconds.")
                time.sleep(wait)
            else:
                print(f"HTTP error occurred: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
        
    print(f"Failed to fetch data for start={start} after 5 attempts")
    return []

def get_total_results():
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity"))',
        'rows': 0
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests_retry_session().get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()['response']['numFound']
    else:
        print(f"Error: {response.status_code}")
        return 0

base_url = "https://api.iatistandard.org/datastore/activity/select"
total_results = get_total_results()
all_activities = []

print(f"Total results to fetch: {total_results}")

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor: 
    futures = [executor.submit(fetch_page, i) for i in range(0, total_results, 1000)]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        all_activities.extend(future.result())
        print(f"Fetched page {i+1}/{len(futures)}")
        time.sleep(1) 

df_activity_1 = pd.DataFrame(all_activities)
print(df_activity_1.head())
print(f"Total activities fetched: {len(df_activity_1)}")

Total results to fetch: 27829
Fetched page 1/28
Fetched page 2/28
Fetched page 3/28
Fetched page 4/28
Fetched page 5/28
Fetched page 6/28
Fetched page 7/28
Fetched page 8/28
Fetched page 9/28
Fetched page 10/28
Fetched page 11/28
Fetched page 12/28
Fetched page 13/28
Fetched page 14/28
Fetched page 15/28
Fetched page 16/28
Fetched page 17/28
Fetched page 18/28
Fetched page 19/28
Fetched page 20/28
Fetched page 21/28
Fetched page 22/28
Fetched page 23/28
Fetched page 24/28
Fetched page 25/28
Fetched page 26/28
Fetched page 27/28
Fetched page 28/28
      sector_code                 iati_identifier  \
0         [72040]                NZ-1-ACT-0103019   
1  [DR, 2, 72010]  NL-KVK-40409352-PRJ13-108-0004   
2           [720]             GB-CHC-220949-P8501   
3         [31120]             XM-DAC-41301-745086   
4             NaN        US-GOV-1-72068523IO00003   

                                     title_narrative activity_date_type  \
0              [2022 Horn of Africa Food Insecurity] 

In [4]:
df_activity_1

,sector_code,iati_identifier,title_narrative,activity_date_type,description_narrative,activity_date_iso_date,recipient_country_code
0,[72040],NZ-1-ACT-0103019,[2022 Horn of Africa Food Insecurity],"[1, 2, 3]",[Humanitarian contributions to support the int...,"[2027-02-01T00:00:00Z, 2022-06-21T00:00:00Z, 2...","[ET, KE, SO, SS]"
1,"[DR, 2, 72010]",NL-KVK-40409352-PRJ13-108-0004,[2022 Food insecurity EA Niger],"[2, 3]",[Niger is currently being affected by the wors...,"[2022-05-08T00:00:00Z, 2023-05-07T00:00:00Z]",[NE]
2,[720],GB-CHC-220949-P8501,[Mauritania Food Insecurity 2020],"[2, 4]",[Mauritania is currently facing a difficult fo...,"[2020-06-20T00:00:00Z, 2021-06-14T00:00:00Z]",[MR]
3,[31120],XM-DAC-41301-745086,[Supporting Food Security Preparedness and Res...,"[2, 3]",[To achieve higher levels of food security in ...,"[2024-02-05T00:00:00Z, 2025-02-28T00:00:00Z]",[SO]
4,NaN,US-GOV-1-72068523IO00003,[Food Insecurity Support],"[1, 2, 3, 4]",[Response to food insecurity linked to global ...,"[2022-10-01T00:00:00Z, 2022-10-01T00:00:00Z, 2...",NaN
...,...,...,...,...,...,...,...
11246,NaN,SE-0-SE-6-14967-14967A0101-H11431,[Not specified 2022 - DRC HUM 2022-2025 Projek...,"[2, 3, 1]","[Founded in 1956, Danish Refugee Council (DRC)...","[2022-03-31T00:00:00Z, 2027-03-31T00:00:00Z, 2...",NaN
11247,NaN,SE-0-SE-6-14967-14967A0101-H11789,[Colombia 2024 - DRC HUM 2022-2025 Projekt och...,"[2, 3, 1]",[Humanitära enheten kommer under 2022 att ge s...,"[2022-03-31T00:00:00Z, 2027-03-31T00:00:00Z, 2...",NaN
11248,NaN,SE-0-SE-6-14967-14967A0101-H11790,[Yemen 2024 - DRC HUM 2022-2025 Projekt och me...,"[2, 3, 1]","[Founded in 1956, Danish Refugee Council (DRC)...","[2022-03-31T00:00:00Z, 2027-03-31T00:00:00Z, 2...",NaN
11249,NaN,SE-0-SE-6-14967-14967A0101-H11792,[Myanmar 2024 - DRC HUM 2022-2025 Projekt och ...,"[2, 3, 1]",[Humanitära enheten kommer under 2022 att ge s...,"[2022-03-31T00:00:00Z, 2027-03-31T00:00:00Z, 2...",NaN


Activity/XML

In [5]:
# Function to handle retry logic for requests
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# Function to fetch a single page of data in XML format with a persistent session
def fetch_page_xml(start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity"))',
        'fq': 'activity_date_iso_date:[2021-01-01T00:00:00Z TO *]',
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    response = requests_retry_session().get(xml_base_url, headers=headers, params=params, timeout=30)
    if response.status_code == 200:
        return response.content  # Return raw XML content
    else:
        response.raise_for_status()

# Function to parse the XML data and extract relevant information
def parse_xml_data(xml_data):
    root = ET.fromstring(xml_data)
    activities = []
    for activity in root.findall('.//iati-activity'):
        iati_id = activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None
        title = activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None
        description = activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None
        
        sector_element = activity.find('.//sector/code')
        sector = sector_element.attrib.get('code') if sector_element is not None else None
        
        date_element = activity.find('.//activity-date[@type="2"]')  # Type "2" is the actual start date
        date = date_element.attrib.get('iso-date') if date_element is not None else None
        
        country_element = activity.find('.//recipient-country')
        country = country_element.attrib.get('code') if country_element is not None else None
        
        activities.append({
            'iati_identifier': iati_id,
            'title_narrative': title,
            'description_narrative': description,
            'sector_code': sector,
            'activity_date_iso_date': date,
            'recipient_country_code': country
        })
    return activities

# Function to fetch all records with pagination
def fetch_all_pages():
    all_activities = []
    start = 0
    while True:
        xml_data = fetch_page_xml(start=start)
        activities = parse_xml_data(xml_data)
        if not activities:  # If no activities are returned, break the loop
            break
        all_activities.extend(activities)
        start += 1000  # Move to the next page
    return all_activities

# Example execution
xml_base_url = "https://api.iatistandard.org/datastore/activity/iati"  # Updated URL for XML

all_activities = fetch_all_pages()

df_activity_xml = pd.DataFrame(all_activities)
print(f"Total activities fetched: {len(df_activity_xml)}")
print(df_activity_xml.head())


Total activities fetched: 11767
                  iati_identifier  \
0                NZ-1-ACT-0103019   
1  NL-KVK-40409352-PRJ13-108-0004   
2             GB-CHC-220949-P8501   
3             XM-DAC-41301-745086   
4        US-GOV-1-72068523IO00003   

                                     title_narrative  \
0                2022 Horn of Africa Food Insecurity   
1                      2022 Food insecurity EA Niger   
2                    Mauritania Food Insecurity 2020   
3  Supporting Food Security Preparedness and Resi...   
4                            Food Insecurity Support   

                               description_narrative sector_code  \
0  Humanitarian contributions to support the inte...        None   
1  Niger is currently being affected by the worst...        None   
2  Mauritania is currently facing a difficult foo...        None   
3  To achieve higher levels of food security in t...        None   
4  Response to food insecurity linked to global p...        None   


In [7]:
df_activity_xml

,iati_identifier,title_narrative,description_narrative,sector_code,activity_date_iso_date,recipient_country_code
0,NZ-1-ACT-0103019,2022 Horn of Africa Food Insecurity,Humanitarian contributions to support the inte...,None,2022-06-21,ET
1,NL-KVK-40409352-PRJ13-108-0004,2022 Food insecurity EA Niger,Niger is currently being affected by the worst...,None,2022-05-08,NE
2,GB-CHC-220949-P8501,Mauritania Food Insecurity 2020,Mauritania is currently facing a difficult foo...,None,2020-06-20,MR
3,XM-DAC-41301-745086,Supporting Food Security Preparedness and Resi...,To achieve higher levels of food security in t...,None,2024-02-05,SO
4,US-GOV-1-72068523IO00003,Food Insecurity Support,Response to food insecurity linked to global p...,None,2022-10-01,MR
...,...,...,...,...,...,...
11762,SE-0-SE-6-10462-10462A0106-H10561,South Sudan Crisis 2022 - Swedish Mission Coun...,Den 14 mars 2017 tog Sidas humanitära enhet be...,None,2017-03-29,SS
11763,SE-0-SE-6-10462-10462A0106-H10562,Syria crisis 2022 - Swedish Mission Council- u...,The proposed programme by the Swedish Mission ...,None,2017-03-29,SY
11764,SE-0-SE-6-10462-10462A0106-H10563,Method/Capacity/Other Support 2022 - Swedish M...,Den 14 mars 2017 tog Sidas humanitära enhet be...,None,2017-03-29,None
11765,SE-0-SE-6-10462-10462A0106-H10565,Method/Capacity/Other Support 2022 - Swedish M...,The proposed programme by the Swedish Mission ...,None,2017-03-29,None


Activity/XML considering SDG tag code 2

OR sdg tag2

In [9]:


def requests_retry_session(session=None, retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504)):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page_xml(session, start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity")) OR tag_vocabulary:2',
        'fq': 'activity_date_iso_date:[2021-01-01T00:00:00Z TO *]',
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    response = session.get(xml_base_url, headers=headers, params=params, timeout=30)
    if response.status_code == 200:
        return response.content  
    else:
        response.raise_for_status()

def parse_xml_data(xml_data):
    root = ET.fromstring(xml_data)
    activities = []
    for activity in root.findall('.//iati-activity'):
        iati_id = activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None
        title = activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None
        description = activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None
        
        sector_element = activity.find('.//sector/code')
        sector = sector_element.attrib.get('code') if sector_element is not None else None
        
        date_element = activity.find('.//activity-date[@type="2"]')  
        date = date_element.attrib.get('iso-date') if date_element is not None else None
        
        country_element = activity.find('.//recipient-country')
        country = country_element.attrib.get('code') if country_element is not None else None
        
        # Extract SDG tags
        sdg_tags = activity.findall(".//tag[@vocabulary='2']")
        sdg_codes = [tag.get('code') for tag in sdg_tags]
        
        activities.append({
            'iati_identifier': iati_id,
            'title_narrative': title,
            'description_narrative': description,
            'sector_code': sector,
            'activity_date_iso_date': date,
            'recipient_country_code': country,
            'sdg_tags': sdg_codes  
        })
    return activities

# Function to fetch all records with pagination
def fetch_all_records():
    session = requests_retry_session()  # Create a persistent session
    all_activities = []
    start = 0
    
    while True:
        xml_data = fetch_page_xml(session, start)
        activities = parse_xml_data(xml_data)
        if not activities:  # If no more activities are returned, exit the loop
            break
        all_activities.extend(activities)
        start += 1000  
    
    return all_activities


xml_base_url = "https://api.iatistandard.org/datastore/activity/iati"  

all_activities = fetch_all_records()
df_activity_xml_sdg = pd.DataFrame(all_activities)
print(f"Total activities fetched: {len(df_activity_xml_sdg )}")


Total activities fetched: 64669


In [10]:
df_activity_xml_sdg

,iati_identifier,title_narrative,description_narrative,sector_code,activity_date_iso_date,recipient_country_code,sdg_tags
0,NZ-1-ACT-0103019,2022 Horn of Africa Food Insecurity,Humanitarian contributions to support the inte...,None,2022-06-21,ET,[]
1,NL-KVK-40409352-PRJ13-108-0004,2022 Food insecurity EA Niger,Niger is currently being affected by the worst...,None,2022-05-08,NE,[]
2,GB-CHC-220949-P8501,Mauritania Food Insecurity 2020,Mauritania is currently facing a difficult foo...,None,2020-06-20,MR,[]
3,XM-DAC-41301-745086,Supporting Food Security Preparedness and Resi...,To achieve higher levels of food security in t...,None,2024-02-05,SO,[]
4,US-GOV-1-72068523IO00003,Food Insecurity Support,Response to food insecurity linked to global p...,None,2022-10-01,MR,[]
...,...,...,...,...,...,...,...
64664,NO-BRC-971277882-SOM-16/0002,Somalia Stability Fund\n\t\t,The planned impact is to enhance Stability in ...,None,2016-12-19,SO,[16]
64665,XM-DAC-6-4-012387-01-0,VOLUNTARY CONTRIBUTION 2021 UN-WOMEN,Voluntary contribution to UN WOMEN that is the...,None,2021-07-08,None,[5]
64666,FI-3-2020-2010100062,F.a: The development cooperation programme of ...,\nThe development cooperation programme of Dis...,None,2010-05-28,None,[11]
64667,XM-DAC-6-4-012352-01-0,VOLUNTARY CONTRIBUTION 2021 GLOBAL PARTNERSHIP...,Voluntary contribution for 2021 through the Gl...,None,2021-08-09,None,[4]


And sdg tag2

In [11]:
def requests_retry_session(session=None, retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504)):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page_xml(session, start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity")) AND tag_vocabulary:2',
        'fq': 'activity_date_iso_date:[2021-01-01T00:00:00Z TO *]',
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    response = session.get(xml_base_url, headers=headers, params=params, timeout=30)
    if response.status_code == 200:
        return response.content  
    else:
        response.raise_for_status()

def parse_xml_data(xml_data):
    root = ET.fromstring(xml_data)
    activities = []
    for activity in root.findall('.//iati-activity'):
        iati_id = activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None
        title = activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None
        description = activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None
        
        sector_element = activity.find('.//sector/code')
        sector = sector_element.attrib.get('code') if sector_element is not None else None
        
        date_element = activity.find('.//activity-date[@type="2"]')  
        date = date_element.attrib.get('iso-date') if date_element is not None else None
        
        country_element = activity.find('.//recipient-country')
        country = country_element.attrib.get('code') if country_element is not None else None
        
        # Extract SDG tags
        sdg_tags = activity.findall(".//tag[@vocabulary='2']")
        sdg_codes = [tag.get('code') for tag in sdg_tags]
        
        activities.append({
            'iati_identifier': iati_id,
            'title_narrative': title,
            'description_narrative': description,
            'sector_code': sector,
            'activity_date_iso_date': date,
            'recipient_country_code': country,
            'sdg_tags': sdg_codes  
        })
    return activities

# Function to fetch all records with pagination
def fetch_all_records():
    session = requests_retry_session()  # Create a persistent session
    all_activities = []
    start = 0
    
    while True:
        xml_data = fetch_page_xml(session, start)
        activities = parse_xml_data(xml_data)
        if not activities:  # If no more activities are returned, exit the loop
            break
        all_activities.extend(activities)
        start += 1000  
    
    return all_activities


xml_base_url = "https://api.iatistandard.org/datastore/activity/iati"  

all_activities = fetch_all_records()
df_activity_xml_and_sdg = pd.DataFrame(all_activities)



In [12]:
df_activity_xml_and_sdg

,iati_identifier,title_narrative,description_narrative,sector_code,activity_date_iso_date,recipient_country_code,sdg_tags
0,XI-IATI-EC_INTPA-2020-PCC-412348,Strengthening Food Security Statistics at coun...,<p>This Project will strengthen the capacity o...,None,2020-11-27,None,"[10, 2, 5]"
1,DAC-1601-INV-041387,Emergency Relief to Food Insecurity in Madagascar,to provide humanitarian aid to people affected...,None,2021-12-06,MG,"[1, 2]"
2,XM-DAC-47015-22403_Window3_USA-USAID-UnitedSta...,Grant: FOOD INSECURITY WARNING (Window 3),"A $500,000 Window 3 grant from USA - USAID-Uni...",None,2022-10-01,AF,[2]
3,XM-DAC-47015-22404_Window3_USA-USAID-UnitedSta...,Grant: ACUTE FOOD INSECURITY (Window 3),"A $1,000,000 Window 3 grant from USA - USAID-U...",None,2022-10-01,AF,[2]
4,DAC-1601-INV-038770,Emergency Response to Food Insecurity in Burki...,to provide humanitarian aid to people affected...,None,2021-09-22,BF,"[1, 5, 2]"
...,...,...,...,...,...,...,...
1576,NL-KVK-27108436-A-06499-02:AF,DEC Coronavirus Appeal fund,Oxfam has prepared the COVID-19 Response Strat...,None,2020-06-14,AF,[1]
1577,NL-KVK-27108436-A-06499-03:AF,CO-DEC Coronavirus Appeal fund,Oxfam has prepared the COVID-19 Response Strat...,None,2020-06-14,AF,[1]
1578,XM-DAC-6-4-012590-03-5,Let's feed the future! Nutritional safety acti...,The project has the general objective of Contr...,None,2022-06-17,BF,[2]
1579,XM-DAC-6-4-012590-01-7,omen and people with disabilities in action fo...,The project aims to strengthen food and nutrit...,None,2022-06-17,NE,"[2, 6]"
